Import packages and processed run metrics

In [13]:
# packages
import os
import re
import numpy as np
import pandas as pd

# source
helper = __import__('00_helper')

# control
project_names = [
    #'femnist--s02', 'sst2', 'pubmed',
    'cifar--alpha5.0', 'cifar--alpha0.5', 'cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

df = helper.load_data(project_names)


input datasets:
	 (520, 36)
	 (520, 36)
	 (520, 36)
all runs: (1560, 35)


Filter data to runs of interest
Apply summary function to filtered dataset

In [14]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches

subset_df = subset_df.loc[(subset_df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', subset_df.shape)

subset_df = subset_df.loc[(subset_df.K < 6) | (subset_df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', subset_df.shape)

subset_df = subset_df.loc[((10 * subset_df.beta).astype('Int64') % 2 == 0) | (subset_df.method != 'exact')]
print('\t reduce beta grid for decay:', subset_df.shape)

## column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if (
        not re.search('^sys', name)
        and not re.search('f1', name)
        and not re.search('loss', name)
    )
]]

input dataset: (1560, 35)
	 reduce to 3 or fewer local update steps: (1230, 35)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (1230, 35)
	 reduce beta grid for decay: (1080, 35)


In [15]:
## get best runs for each group
# regardless of finetuning
best_runs = subset_df.loc[
    subset_df.groupby(['dataset', 'method', 'finetune']) \
    [validation_metric].idxmax()  # return index of max validation metric
]

# treat finetuning groups as seperate
ft_yes = best_runs.loc[best_runs.finetune == 1]
ft_no = best_runs.loc[best_runs.finetune == 0]

For all best runs, regardless of finetuning, produce summaries

In [16]:
# For existing users, FT since sufficient data
seen_best_runs = ft_yes[[
    name for name in ft_yes.columns
    if (
        not re.search('weighted', name)
        and not re.search('unseen', name)
    )
    or (
        not re.search('Results', name)
        or re.search('val', name)
    )
]]
helper.runs_to_latex(seen_best_runs, 'seen--ft_yes')

(seen_run_summary, seen_metric_summary, seen_rank_summary) = helper.process_run_metrics(seen_best_runs)
print('all runs:', seen_run_summary.shape)
print('all metrics:', seen_metric_summary.shape)

all runs: (4, 4)
all metrics: (3, 4)


In [17]:
# run summary
seen_run_summary.sort_values(by='value', ascending=False)

,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
exact,1,4,4,1,9
fedavg,1,4,3,2,9
pfedme,1,5,0,1,6
fedem,1,0,0,3,3


View processed summaries

In [18]:
# metrics summary
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
seen_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results/test_acc_bottom_decile,2,1,0,3
Results/test_acc_std,2,1,0,3
Results/test_acc,0,2,1,3


Now seperately, based on finetuning status, repeat the above summary

In [19]:
unseen_best_runs = best_runs[[
    name for name in ft_yes.columns
    if (
               not re.search('weighted', name)
               and re.search('unseen', name)
       )
       or (
               not re.search('Results', name)
               or re.search('val', name)
       )
]]

In [20]:
ft_yes = unseen_best_runs.loc[unseen_best_runs.finetune == 1]
ft_no = unseen_best_runs.loc[unseen_best_runs.finetune == 0]

(yes_run_summary, yes_metric_summary, _) = helper.process_run_metrics(ft_yes)
helper.runs_to_latex(ft_yes, 'unseen--ft_yes')

(no_run_summary, no_metric_summary, _) = helper.process_run_metrics(ft_no)
helper.runs_to_latex(ft_no, 'unseen--ft_no')

In [21]:
# run summary
yes_run_summary.sort_values(by='value', ascending=False)

,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
exact,1,5,2,1,8
fedavg,1,6,1,0,7
fedem,1,1,2,3,6
pfedme,1,2,0,4,6


In [22]:
# metrics summary
yes_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc_bottom_decile,2,0,1,3
Results_unseen/test_acc_std,1,2,0,3
Results_unseen/test_acc,2,0,0,2


In [23]:
# run summary
no_run_summary.sort_values(by='value', ascending=False)

,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
fedavg,0,4,2,3,9
exact,0,5,2,1,8
fedem,0,2,1,5,8
ditto,0,1,0,0,1
pfedme,0,0,1,0,1


In [24]:
# metrics summary
no_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc,2,1,0,3
Results_unseen/test_acc_bottom_decile,2,0,1,3
Results_unseen/test_acc_std,1,1,0,2
